<a href="https://colab.research.google.com/github/lavyay2/-my-projects-for-intern/blob/main/Copy_of_Local_Food_Wastage_Management_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Local Food Wastage Management System**



### **Business Problem**
Every year, tons of edible food are wasted while millions go hungry.
Restaurants, grocery stores, and households often have surplus food, but lack an efficient way to connect with NGOs or individuals in need.

**This project builds a SQL-powered Local Food Wastage Management System that:**
- Stores surplus food provider and receiver data in a structured SQL database.
- Tracks claims and distributions in real time.
- Analyzes patterns to improve efficiency and reduce waste.

---

### **Why This Project is Important**
- **Social Impact:** Directly reduces hunger by redistributing surplus food.
- **Waste Reduction:** Minimizes the environmental and economic cost of wasted food.
- **Data-Driven:** SQL queries help identify top contributors, busiest locations, and demand trends.
- **Scalable:** Can be expanded to cover multiple cities and integrate with live geolocation systems.
**bold text**

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
# app.py
import streamlit as st
import pandas as pd
import sqlite3

#  PAGE SETTINGS
st.set_page_config(page_title="Local Food Wastage Management – SQL Analysis", layout="wide")
st.title("Local Food Wastage Management – SQL Analysis in Python")

#  DATA CLEANING
def clean_df(df):
    df = df.drop_duplicates()
    df.columns = df.columns.str.strip()
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.strip().fillna("Unknown")
    for col in df.select_dtypes(include='number'):
        df[col] = df[col].fillna(0)
    df = df.dropna(how='all')
    return df

# Load raw CSVs (update names as per your files)
providers = pd.read_csv("/providers_data .csv")
receivers = pd.read_csv("/receivers_clean.csv")
food_listings = pd.read_csv("/food_listings_clean.csv")
claims = pd.read_csv("/claims_data.csv")

# Clean each
providers = clean_df(providers)
receivers = clean_df(receivers)
food_listings = clean_df(food_listings)
claims = clean_df(claims)

# Show cleaned samples
st.subheader(" Cleaned Data Samples")
st.write("**Providers**")
st.dataframe(providers.head())
st.write("**Receivers**")
st.dataframe(receivers.head())
st.write("**Food Listings**")
st.dataframe(food_listings.head())
st.write("**Claims**")
st.dataframe(claims.head())

# ---------------------- STORE IN SQLITE ----------------------
conn = sqlite3.connect("food_waste_clean.db")
providers.to_sql("providers", conn, if_exists="replace", index=False)
receivers.to_sql("receivers", conn, if_exists="replace", index=False)
food_listings.to_sql("food_listings", conn, if_exists="replace", index=False)
claims.to_sql("claims", conn, if_exists="replace", index=False)

def run_query(query):
    return pd.read_sql(query, conn)

# QUERIES
queries = [
    ("Providers & Receivers per City", "Shows cities with most providers & receivers.", """
     SELECT p.City,
            COUNT(DISTINCT p.Provider_ID) AS Providers,
            COUNT(DISTINCT r.Receiver_ID) AS Receivers
     FROM providers p
     LEFT JOIN receivers r ON p.City = r.City
     GROUP BY p.City;
     """, "High activity cities can be prioritized for collection and distribution."),

    ("Top Provider Type by Quantity", "Identifies provider types contributing most food.", """
     SELECT Provider_Type, SUM(Quantity) AS Total_Quantity
     FROM food_listings
     GROUP BY Provider_Type
     ORDER BY Total_Quantity DESC;
     """, "Focus engagement efforts on top-yield provider categories."),

    ("Contact Info in Delhi", "Lists providers in Delhi with contact details.", """
     SELECT Name, Contact
     FROM providers
     WHERE City = 'Delhi';
     """, "Enables quick communication for food pickups."),

    ("Top Receivers by Claims", "Most active food claimants.", """
     SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
     FROM receivers r
     JOIN claims c ON r.Receiver_ID = c.Receiver_ID
     GROUP BY r.Name
     ORDER BY Total_Claims DESC;
     """, "These receivers can be prioritized for urgent distributions."),

    ("Total Food Available", "Overall quantity available in the system.", """
     SELECT SUM(Quantity) AS Total_Quantity_Available
     FROM food_listings;
     """, "Helps plan if supply meets demand."),

    ("City with Most Listings", "City with the highest number of food listings.", """
     SELECT Location AS City, COUNT(Food_ID) AS Total_Listings
     FROM food_listings
     GROUP BY Location
     ORDER BY Total_Listings DESC;
     """, "Indicates high supply zones."),

    ("Most Common Food Types", "Most frequently donated food categories.", """
     SELECT Food_Type, COUNT(*) AS Count_Type
     FROM food_listings
     GROUP BY Food_Type
     ORDER BY Count_Type DESC;
     """, "Supports planning for different dietary needs."),

    ("Claims per Food Item", "Number of claims made for each food item.", """
     SELECT f.Food_Name, COUNT(c.Claim_ID) AS Total_Claims
     FROM food_listings f
     LEFT JOIN claims c ON f.Food_ID = c.Food_ID
     GROUP BY f.Food_Name
     ORDER BY Total_Claims DESC;
     """, "Shows which items are in highest demand."),

    ("Most Successful Providers", "Providers with the most completed claims.", """
     SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
     FROM providers p
     JOIN food_listings f ON p.Provider_ID = f.Provider_ID
     JOIN claims c ON f.Food_ID = c.Food_ID
     WHERE c.Status = 'Completed'
     GROUP BY p.Name
     ORDER BY Successful_Claims DESC;
     """, "Recognize and encourage these providers."),

    ("Claim Status Percentages", "Distribution of claim statuses.", """
     SELECT Status,
            ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims), 2) AS Percentage
     FROM claims
     GROUP BY Status;
     """, "Shows operational efficiency."),

    ("Average Quantity per Receiver", "Average quantity claimed per receiver.", """
     SELECT r.Name, AVG(f.Quantity) AS Avg_Quantity
     FROM receivers r
     JOIN claims c ON r.Receiver_ID = c.Receiver_ID
     JOIN food_listings f ON c.Food_ID = f.Food_ID
     GROUP BY r.Name;
     """, "Helps plan allocations."),

    ("Most Claimed Meal Type", "Meal types with highest claim counts.", """
     SELECT Meal_Type, COUNT(*) AS Total_Claims
     FROM food_listings f
     JOIN claims c ON f.Food_ID = c.Food_ID
     GROUP BY Meal_Type
     ORDER BY Total_Claims DESC;
     """, "Guides sourcing of popular meal types."),

    ("Total Quantity by Provider", "Total donated quantity by each provider.", """
     SELECT p.Name, SUM(f.Quantity) AS Total_Donated
     FROM providers p
     JOIN food_listings f ON p.Provider_ID = f.Provider_ID
     GROUP BY p.Name
     ORDER BY Total_Donated DESC;
     """, "Acknowledges top donors."),

    ("Top 5 Cities by Completed Claims", "Cities with highest successful claims.", """
     SELECT f.Location AS City, COUNT(*) AS Completed_Claims
     FROM claims c
     JOIN food_listings f ON c.Food_ID = f.Food_ID
     WHERE c.Status = 'Completed'
     GROUP BY f.Location
     ORDER BY Completed_Claims DESC
     LIMIT 5;
     """, "These cities are models for effective redistribution."),

    ("Month-wise Claims Trend", "Trends in claims by month.", """
     SELECT strftime('%Y-%m', Timestamp) AS Month, COUNT(*) AS Total_Claims
     FROM claims
     GROUP BY Month
     ORDER BY Month;
     """, "Helps plan for seasonal peaks.")
]

#  DISPLAY RESULTS
st.subheader(" SQL Query Outputs")
for title, purpose, sql, insight in queries:
    st.markdown(f"### {title}")
    st.caption(f"Purpose: {purpose}")
    st.code(sql, language="sql")
    df = run_query(sql)
    st.dataframe(df)
    st.success(f" {insight}")



2025-08-13 06:10:44.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 06:10:44.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 06:10:44.020 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 06:10:44.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 06:10:44.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 06:10:44.064 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 06:10:44.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 06:10:44.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

 **Conclusion**
This SQL analysis identifies:
- Key cities and providers in food redistribution  
- Popular food items and meal types  
- Seasonal demand patterns  
- Operational efficiency metrics  

**Impact:**  
These insights help reduce food waste, improve distribution, and ensure surplus food reaches those in need